In [1]:
!pip install "/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"
!pip install "/kaggle/input/autocorrect/autocorrect-2.6.1.tar"
!pip install "/kaggle/input/pyphen-0100/Pyphen-0.10.0-py3-none-any.whl"

Processing /kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl
Processing /kaggle/input/autocorrect/autocorrect-2.6.1.tar
  Preparing metadata (setup.py) ... - \ done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622383 sha256=2ed9013197fcc7e033127b8f038b21f2b06567b51b85fb36773772477a9f4baa
  Stored in directory: /root/.cache/pip/wheels/db/69/42/0fb0421d2fe70d195a04665edc760cfe5fd341d7bb8d8e0aaa
Successfully built autocorrect
Processing /kaggle/input/pyphen-0100/Pyphen-0.10.0-py3-none-any.whl


In [2]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold, GroupKFold
from transformers import AutoTokenizer,TrainingArguments, Trainer, AutoModelForSequenceClassification,DataCollatorWithPadding, get_linear_schedule_with_warmup
import numpy as np
import shutil
import torch.nn as nn
import json
import torch
from datasets import Dataset
from torch.optim import AdamW
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")
import nltk
from collections import Counter
import sys
sys.path.append('../input/sentence-transformers-offline-install/sentence-transformers')
from sentence_transformers import SentenceTransformer, util
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from spellchecker import SpellChecker
import matplotlib.pyplot as plt
STOP_WORDS = set(stopwords.words('english'))
speller = SpellChecker()
# pd.set_option('display.max_colwidth', None)
import re
import optuna
from typing import List
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import spacy
from autocorrect import Speller
from spellchecker import SpellChecker
from nltk import word_tokenize, pos_tag
from textblob import TextBlob
import pyphen

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
with open('/kaggle/input/commonlit-train/config.json') as json_file:
    config = json.load(json_file)
oof_df = pd.read_csv("/kaggle/input/commonlit-train/oof_df.csv")
tokenizer = AutoTokenizer.from_pretrained(config["model_path"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# train_df = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv")
# train_prompts_df = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv")
# train_df = train_df.merge(train_prompts_df, on="prompt_id")
# train_df['input'] = train_df["prompt_question"] + "[SEP]" + train_df["text"]
# train_df = pd.merge(train_df, oof_df[["student_id","content_pred","wording_pred"]], on="student_id")
# train_df.head()

In [5]:
dic = pyphen.Pyphen(lang='en')
sid = SentimentIntensityAnalyzer()

class Preprocessor:
    def __init__(self, 
                model_name: str,
                ) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.twd = TreebankWordDetokenizer()
        self.STOP_WORDS = set(stopwords.words('english'))
        
        self.spacy_ner_model = spacy.load('en_core_web_sm',)
        self.speller = Speller(lang='en')
        self.spellchecker = SpellChecker() 
        
    def word_overlap_count(self, row):
        """ intersection(prompt_text, text) """        
        def check_is_stop_word(word):
            return word in self.STOP_WORDS
        
        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))
            
    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int) -> int:
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)
        return len(common_ngrams)
    
    def ner_overlap_count(self, row, mode:str):
        model = self.spacy_ner_model
        def clean_ners(ner_list):
            return set([(ner[0].lower(), ner[1]) for ner in ner_list])
        prompt = model(row['prompt_text'])
        summary = model(row['text'])

        if "spacy" in str(model):
            prompt_ner = set([(token.text, token.label_) for token in prompt.ents])
            summary_ner = set([(token.text, token.label_) for token in summary.ents])
        elif "stanza" in str(model):
            prompt_ner = set([(token.text, token.type) for token in prompt.ents])
            summary_ner = set([(token.text, token.type) for token in summary.ents])
        else:
            raise Exception("Model not supported")

        prompt_ner = clean_ners(prompt_ner)
        summary_ner = clean_ners(summary_ner)

        intersecting_ners = prompt_ner.intersection(summary_ner)
        
        ner_dict = dict(Counter([ner[1] for ner in intersecting_ners]))
        
        if mode == "train":
            return ner_dict
        elif mode == "test":
            return {key: ner_dict.get(key) for key in self.ner_keys}

    
    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def spelling(self, text):
        
        wordlist=text.split()
        amount_miss = len(list(self.spellchecker.unknown(wordlist)))

        return amount_miss
    ##############################################################################################
    def calculate_pos_ratios(self, text):
        pos_tags = pos_tag(nltk.word_tokenize(text))
        pos_counts = Counter(tag for word, tag in pos_tags)
        total_words = len(pos_tags)
        ratios = {tag: count / total_words for tag, count in pos_counts.items()}
        return ratios
    
    def calculate_unique_words(self, text):
        unique_words = set(text.split())
        return len(unique_words)
    
    def calculate_keyword_density(self, row):
        keywords = set(row['prompt_text'].split())
        text_words = row['text'].split()
        keyword_count = sum(1 for word in text_words if word in keywords)
        return keyword_count / len(text_words)
    
    def calculate_sentiment_scores_prompt(self, text):
        sentiment_scores = sid.polarity_scores(text)
        return sentiment_scores
    
    def sentiment_analysis(self, text):
        analysis = TextBlob(text)
        return analysis.sentiment.polarity, analysis.sentiment.subjectivity
    
    def count_syllables(self,word):
        hyphenated_word = dic.inserted(word)
        return len(hyphenated_word.split('-'))

    def flesch_reading_ease_manual(self,text):
        total_sentences = len(TextBlob(text).sentences)
        total_words = len(TextBlob(text).words)
        total_syllables = sum(self.count_syllables(word) for word in TextBlob(text).words)

        if total_sentences == 0 or total_words == 0:
            return 0

        flesch_score = 206.835 - 1.015 * (total_words / total_sentences) - 84.6 * (total_syllables / total_words)
        return flesch_score
    
    def flesch_kincaid_grade_level(self, text):
        total_sentences = len(TextBlob(text).sentences)
        total_words = len(TextBlob(text).words)
        total_syllables = sum(self.count_syllables(word) for word in TextBlob(text).words)

        if total_sentences == 0 or total_words == 0:
            return 0

        fk_grade = 0.39 * (total_words / total_sentences) + 11.8 * (total_syllables / total_words) - 15.59
        return fk_grade
    
    
    def gunning_fog(self, text):
        total_sentences = len(TextBlob(text).sentences)
        total_words = len(TextBlob(text).words)
        complex_words = sum(1 for word in TextBlob(text).words if self.count_syllables(word) > 2)

        if total_sentences == 0 or total_words == 0:
            return 0

        fog_index = 0.4 * ((total_words / total_sentences) + 100 * (complex_words / total_words))
        return fog_index
    
    def calculate_sentiment_scores(self,text):
        sentiment_scores = sid.polarity_scores(text)
        return sentiment_scores
    
    def count_difficult_words(self, text, syllable_threshold=3):
        words = TextBlob(text).words
        difficult_words_count = sum(1 for word in words if self.count_syllables(word) >= syllable_threshold)
        return difficult_words_count
    
    ##############################################################################################
    
    def add_spelling_dictionary(self, tokens: List[str]) -> List[str]:
        """dictionary update for pyspell checker and autocorrect"""
        self.spellchecker.word_frequency.load_words(tokens)
        self.speller.nlp_data.update({token:1000 for token in tokens})
    
    def run(self, 
            prompts: pd.DataFrame,
            summaries:pd.DataFrame,
            mode:str
        ) -> pd.DataFrame:
        
        # before merge preprocess
        prompts["prompt_length"] = prompts["prompt_text"].apply(
            lambda x: len(word_tokenize(x))
        )
        prompts["prompt_tokens"] = prompts["prompt_text"].apply(
            lambda x: word_tokenize(x)
        )

        summaries["summary_length"] = summaries["text"].apply(
            lambda x: len(word_tokenize(x))
        )
        summaries["summary_tokens"] = summaries["text"].apply(
            lambda x: word_tokenize(x)
        )
        
        # Add prompt tokens into spelling checker dictionary
        prompts["prompt_tokens"].apply(
            lambda x: self.add_spelling_dictionary(x)
        )
        
#         from IPython.core.debugger import Pdb; Pdb().set_trace()
        # fix misspelling
        summaries["fixed_summary_text"] = summaries["text"].progress_apply(
            lambda x: self.speller(x)
        )
        
        # count misspelling
        summaries["splling_err_num"] = summaries["text"].progress_apply(self.spelling)
        
        # merge prompts and summaries
        input_df = summaries.merge(prompts, how="left", on="prompt_id")

        # after merge preprocess
        # input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']
        
        input_df['word_overlap_count'] = input_df.progress_apply(self.word_overlap_count, axis=1)
        input_df['bigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence,args=(2,), axis=1 
        )
        input_df['bigram_overlap_ratio'] = input_df['bigram_overlap_count'] / (input_df['summary_length'] - 1)
        
        input_df['trigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )
        input_df['trigram_overlap_ratio'] = input_df['trigram_overlap_count'] / (input_df['summary_length'] - 2)
        
        input_df['quotes_count'] = input_df.progress_apply(self.quotes_count, axis=1)
        
        ##############################################################################################
        input_df['pos_ratios'] = input_df['text'].apply(self.calculate_pos_ratios)
        input_df['pos_mean'] = input_df['pos_ratios'].apply(lambda x: np.mean(list(x.values())))
        input_df['unique_word_count'] = input_df['text'].apply(self.calculate_unique_words)
        input_df['keyword_density'] = input_df.apply(self.calculate_keyword_density, axis=1)
        input_df['jaccard_similarity'] = input_df.apply(lambda row: len(set(word_tokenize(row['prompt_text'])) & set(word_tokenize(row['text']))) / len(set(word_tokenize(row['prompt_text'])) | set(word_tokenize(row['text']))), axis=1)
        input_df['sentiment_scores_prompt'] = input_df['prompt_text'].apply(lambda x:self.calculate_sentiment_scores_prompt(x))
        sentiment_columns_prompt = pd.DataFrame(list(input_df['sentiment_scores_prompt']))
        sentiment_columns_prompt.columns = [col +'_prompt' for col in sentiment_columns_prompt.columns]
        input_df = pd.concat([input_df, sentiment_columns_prompt], axis=1)
        input_df['gunning_fog'] = input_df['text'].apply(self.gunning_fog)
        input_df['flesch_kincaid_grade_level'] = input_df['text'].apply(self.flesch_kincaid_grade_level)
        input_df['flesch_reading_ease'] = input_df['text'].apply(self.flesch_reading_ease_manual)
        input_df['count_difficult_words'] = input_df['text'].apply(self.count_difficult_words)
        input_df['vocabulary_richness'] = input_df['text'].apply(lambda x: len(set(x.split())))
        input_df[['sentiment_polarity', 'sentiment_subjectivity']] = input_df['text'].progress_apply(
            lambda x: pd.Series(self.sentiment_analysis(x))
        )
        ##############################################################################################
        
        
        return input_df.drop(columns=["summary_tokens", "prompt_tokens","pos_ratios","sentiment_scores_prompt"])
    
preprocessor = Preprocessor(model_name=config["model_path"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
summaries_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv")
prompts_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv")
train_df = preprocessor.run(prompts_train, summaries_train, mode="train")
train_df = pd.merge(train_df, oof_df[["student_id","content_pred","wording_pred"]], on="student_id")


100%|██████████| 7165/7165 [00:05<00:00, 1304.37it/s]


In [7]:
def create_folds(df, n_folds, strategy):
    df["fold"] = -1
    # just use for spliting the data
    df["bins"] = pd.cut(
        df["content"], bins=n_folds, labels=False
    )
    if strategy == "KFOLD":
        kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=config["random_state"])
        for i, (train , val) in enumerate(kf.split(X=df, y=df["bins"].values)):
            df.loc[val, 'fold'] = i
            
    elif strategy == "GROUPKFOLD":
        gkf = StratifiedGroupKFold(n_splits=n_folds, shuffle=True, random_state=config["random_state"])
        for i, (train , val) in enumerate(gkf.split(X=df, y=df["bins"].values, groups=df["prompt_id"].values)):
            df.loc[val, 'fold'] = i
            
    df = df.drop("bins", axis=1)
    return df

train_df = create_folds(train_df, config["n_folds"], config["strategy"])
train_df["fold"].value_counts()

1    2057
0    2009
2    1996
3    1103
Name: fold, dtype: int64

In [8]:
# targets = ["content","wording"]
# used_cols = [ "wording_pred","text_len","prompt_text_len","word_overlap_counts", "comma_count","semicolon_count","colon_count","fullstop_count","keyword_density", ]
# feature_cols = {
#     "wording": used_cols+sentiment_cols ,
#     "content": used_cols+sentiment_cols
# }

targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text", "fixed_summary_text",
                "prompt_question", "prompt_title", 
                "prompt_text"
               ] + targets

In [9]:
total_features = train_df.drop(columns=drop_columns).shape[1]
print(total_features)

26


In [10]:
train_df.drop(columns=drop_columns)

,summary_length,splling_err_num,prompt_length,word_overlap_count,bigram_overlap_count,bigram_overlap_ratio,trigram_overlap_count,trigram_overlap_ratio,quotes_count,pos_mean,...,compound_prompt,gunning_fog,flesch_kincaid_grade_level,flesch_reading_ease,count_difficult_words,vocabulary_richness,sentiment_polarity,sentiment_subjectivity,content_pred,wording_pred
0,64,5,660,14,4,0.063492,0,0.000000,0,0.052632,...,0.9915,10.034426,6.413238,76.244775,6,51,0.170455,0.334848,-0.093123,0.813397
1,54,2,1076,18,22,0.415094,10,0.192308,0,0.058824,...,-0.9949,10.400000,7.030769,90.964231,0,38,-0.700000,0.666667,-0.856800,-0.436627
2,269,32,625,22,52,0.194030,23,0.086142,2,0.034483,...,0.9283,10.578531,7.480000,76.463164,16,149,0.377778,0.483889,2.654764,2.207916
3,28,5,625,6,6,0.222222,5,0.192308,0,0.076923,...,0.9283,9.800000,4.861000,82.475500,3,19,0.000000,0.000000,-1.049015,-1.023883
4,232,29,660,23,27,0.116883,5,0.021739,4,0.035714,...,0.9915,8.393728,6.352909,75.595915,13,138,0.048203,0.355229,2.184468,2.445339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7160,76,9,1076,14,41,0.546667,35,0.472973,1,0.045455,...,-0.9949,15.018182,11.583030,70.794545,3,49,0.279167,0.400000,-0.099797,-0.009116
7161,49,7,625,6,4,0.083333,1,0.021277,0,0.058824,...,0.9283,7.593798,5.093023,84.077364,2,29,0.216667,0.466667,-0.286430,-0.406556
7162,51,10,625,10,5,0.100000,0,0.000000,0,0.055556,...,0.9283,11.266667,8.028333,80.250000,2,42,0.208333,0.441667,-0.981191,-0.844392
7163,63,5,699,12,4,0.064516,0,0.000000,0,0.052632,...,-0.9969,7.866667,4.680000,96.537740,0,42,0.675000,0.775000,0.017337,0.524402


In [11]:
# model_dict = {}

# for target in targets:
#     models = []
#     for fold in range(config["n_folds"]):
#         X_train = train_df[train_df['fold']!=fold].reset_index(drop=True)[feature_cols[target]]
#         y_train = train_df[train_df['fold']!=fold].reset_index(drop=True)[target]
#         X_val = train_df[train_df['fold']==fold].reset_index(drop=True)[feature_cols[target]]
#         y_val = train_df[train_df['fold']==fold].reset_index(drop=True)[target]
        
#         dtrain = lgb.Dataset(X_train, label=y_train)
#         dval = lgb.Dataset(X_val, label=y_val)
        
#         params = {
#             'boosting_type': 'gbdt',
#             'random_state': 42,
#             'objective': 'regression',
#             'metric': 'rmse',
#             'learning_rate': 0.048,
#             'max_depth': 3,
#             'lambda_l1': 0,
#             'lambda_l2': 0.011,
# #             "num_leaves":10,
# #             "min_data_in_leaf":5
#             "verbose":-1
#         }
        
#         evaluation_results = {}
#         model = lgb.train(params,
#                           num_boost_round=10000,
#                           valid_names=['valid', 'train'],
#                           train_set=dtrain,
#                           valid_sets=[dval,dtrain],
#                           callbacks=[
#                               lgb.early_stopping(stopping_rounds=30, verbose=True),
#                                lgb.log_evaluation(100),
#                               lgb.callback.record_evaluation(evaluation_results)
#                             ],
#                           )
#         models.append(model)
    
#     model_dict[target] = models

In [12]:
# lgb.plot_metric(evaluation_results)

In [13]:
# for model in model_dict["wording"]:
#     lgb.plot_importance(model, max_num_features=13)
#     plt.show()

In [14]:
# rmses = []

# for target in targets:
#     models = model_dict[target]

#     preds = []
#     trues = []
    
#     for fold, model in enumerate(models):
#         X_val = train_df[train_df['fold']==fold].reset_index(drop=True)[feature_cols[target]]
#         y_val = train_df[train_df['fold']==fold].reset_index(drop=True)[target]
#         pred = model.predict(X_val)
#         trues.extend(y_val)
#         preds.extend(pred)
    
#     rmse = np.sqrt(mean_squared_error(trues, preds))
#     print(f"{target}_rmse : {rmse}")
#     rmses = rmses + [rmse]

# print(f"mcrmse : {sum(rmses) / len(rmses)}")

In [15]:
# for k,model_list in model_dict.items():
#     for i,model in enumerate(model_list):
#         model.save_model(f'{k}_fold{i}.txt')

# Optuna

In [16]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [17]:
def objective(trial):
    model_dict = {}
    params = {
                'boosting_type': 'gbdt',
                'random_state': 42,
                'objective': 'regression',
                'metric': 'rmse',
                'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
                'max_depth': trial.suggest_int('max_depth', 2, 10),
                'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 2.5), 
                'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 2.5),
                'num_leaves': trial.suggest_int('num_leaves', 2, 20),
                'subsample': trial.suggest_float('subsample', 0.5, 0.9),
                'min_child_weight': trial.suggest_float('min_child_weight', 0.5, 5), 
                'n_estimators': trial.suggest_int('n_estimators', 50, 350),
                'verbosity': -1,
                'max_depth': trial.suggest_int('max_depth', 2, 10),
                "n_content_features": trial.suggest_int("n_content_features", 18,total_features),
                "n_wording_features": trial.suggest_int("n_wording_features", 18,total_features),
            }

    for target in targets:
        models = []
        fs = SelectKBest(score_func=f_regression, k=params[f"n_{target}_features"])
        fs.fit(train_df.drop(columns=drop_columns), train_df[target])
        selected_features = fs.get_feature_names_out()

        for fold in range(config["n_folds"]):
            X_train = train_df[train_df["fold"] != fold].drop(columns=drop_columns)[selected_features]
            y_train = train_df[train_df["fold"] != fold][target]
            X_val = train_df[train_df["fold"] == fold].drop(columns=drop_columns)[selected_features]
            y_val = train_df[train_df["fold"] == fold][target]
            
            dtrain = lgb.Dataset(X_train, label=y_train)
            dval = lgb.Dataset(X_val, label=y_val)   

            
            model_params = dict((k,v) for k,v in params.items() if k not in ["n_content_features", "n_wording_features"])
            model = lgb.train(model_params,
                              num_boost_round=10000,
                              valid_names=['valid', 'train'],
                              train_set=dtrain,
                              valid_sets=[dval,dtrain],
                              callbacks=[
                                  lgb.early_stopping(stopping_rounds=30, verbose=True),
                                ],
                              )
            models.append(model)

        model_dict[target] = models
            
    rmses = []

    for target in targets:
        models = model_dict[target]
        fs = SelectKBest(score_func=f_regression, k=params[f"n_{target}_features"])
        fs.fit(train_df.drop(columns=drop_columns), train_df[target])
        selected_features = fs.get_feature_names_out()

        preds = []
        trues = []

        for fold, model in enumerate(models):
            X_val = train_df[train_df["fold"] == fold].drop(columns=drop_columns)[selected_features]
            y_val = train_df[train_df["fold"] == fold][target]
            X_val = X_val[fs.get_feature_names_out()]
            pred = model.predict(X_val)
            trues.extend(y_val)
            preds.extend(pred)

        rmse = np.sqrt(mean_squared_error(trues, preds))
        print(f"{target}_rmse : {rmse}")
        rmses = rmses + [rmse]
        
    return sum(rmses)/len(rmses)

In [18]:
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial), n_trials=500)

[I 2023-10-08 19:36:00,721] A new study created in memory with name: no-name-7d3cf180-ca4f-47e2-903b-c78f66521d69


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[109]	train's rmse: 0.388347	valid's rmse: 0.48587
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[159]	train's rmse: 0.407874	valid's rmse: 0.399113
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[54]	train's rmse: 0.427479	valid's rmse: 0.405944
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[159]	train's rmse: 0.393275	valid's rmse: 0.480138
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[159]	train's rmse: 0.491122	valid's rmse: 0.648801
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[159]	train's rmse: 0.53116	valid's rmse: 0.574392
Training until validation scores don't improve for 30 rounds
Ea

[I 2023-10-08 19:36:01,827] Trial 0 finished with value: 0.5090733715147715 and parameters: {'learning_rate': 0.08351516408477345, 'max_depth': 5, 'lambda_l1': 2.6070946513073755e-07, 'lambda_l2': 0.15505421317399062, 'num_leaves': 3, 'subsample': 0.6794526406394932, 'min_child_weight': 2.2856684034256354, 'n_estimators': 159, 'n_content_features': 21, 'n_wording_features': 21}. Best is trial 0 with value: 0.5090733715147715.


wording_rmse : 0.578467219752042
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[131]	train's rmse: 0.410921	valid's rmse: 0.505457
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[260]	train's rmse: 0.415559	valid's rmse: 0.407681
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[260]	train's rmse: 0.408164	valid's rmse: 0.404842
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[260]	train's rmse: 0.39872	valid's rmse: 0.493794
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[260]	train's rmse: 0.488986	valid's rmse: 0.651632
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[260]	train's rmse: 0.525974	valid's rmse: 0.576227
Training until 

[I 2023-10-08 19:36:03,625] Trial 1 finished with value: 0.5154215734983533 and parameters: {'learning_rate': 0.020062854547099388, 'max_depth': 3, 'lambda_l1': 0.0011010337213218225, 'lambda_l2': 9.61977621189864e-07, 'num_leaves': 6, 'subsample': 0.7174805933249575, 'min_child_weight': 0.5643282335170718, 'n_estimators': 260, 'n_content_features': 18, 'n_wording_features': 21}. Best is trial 0 with value: 0.5090733715147715.


...

content_rmse : 0.4369646092041561
wording_rmse : 0.5764833953270795
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[86]	train's rmse: 0.36977	valid's rmse: 0.478956
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[84]	train's rmse: 0.38973	valid's rmse: 0.397975
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[49]	train's rmse: 0.395566	valid's rmse: 0.399993
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[75]	train's rmse: 0.378769	valid's rmse: 0.486157
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[203]	train's rmse: 0.44721	valid's rmse: 0.632147
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[170]	train's rmse: 0.486438	valid's rmse: 0.568142
Training until validation scores don't improve for 

[I 2023-10-08 19:48:24,160] Trial 498 finished with value: 0.5040243024274238 and parameters: {'learning_rate': 0.09999791083538623, 'max_depth': 3, 'lambda_l1': 0.8861267905077829, 'lambda_l2': 2.472523652279562e-07, 'num_leaves': 7, 'subsample': 0.6916818058325384, 'min_child_weight': 2.2148302665213837, 'n_estimators': 260, 'n_content_features': 25, 'n_wording_features': 23}. Best is trial 454 with value: 0.5024218240312192.


content_rmse : 0.4367456472978243
wording_rmse : 0.5713029575570232
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[166]	train's rmse: 0.373664	valid's rmse: 0.485054
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[178]	train's rmse: 0.393972	valid's rmse: 0.4003
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[36]	train's rmse: 0.427612	valid's rmse: 0.404949
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[129]	train's rmse: 0.3863	valid's rmse: 0.47782
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[241]	train's rmse: 0.47298	valid's rmse: 0.646048
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[174]	train's rmse: 0.517253	valid's rmse: 0.566759
Training until validation scores don't

[I 2023-10-08 19:48:25,539] Trial 499 finished with value: 0.506991514612702 and parameters: {'learning_rate': 0.09620739909559137, 'max_depth': 2, 'lambda_l1': 1.2430728652314331, 'lambda_l2': 3.111087792310385e-06, 'num_leaves': 8, 'subsample': 0.6971589748424721, 'min_child_weight': 1.953906497564405, 'n_estimators': 241, 'n_content_features': 26, 'n_wording_features': 23}. Best is trial 454 with value: 0.5024218240312192.


content_rmse : 0.4390921408993675
wording_rmse : 0.5748908883260364


In [19]:
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(yaxis_range=[0.45,0.55])
fig.show()

In [20]:
params = study.best_params
for target in targets:
    fs = SelectKBest(score_func=f_regression, k=params[f"n_{target}_features"])
    fs.fit(train_df.drop(columns=drop_columns), train_df[target])
    selected_features = fs.get_feature_names_out()
    config[f"lightgbm_{target}_features"] = list(selected_features)

In [21]:
config

{'name': 'fixed_summary',
 'random_state': 42,
 'n_folds': 4,
 'model_path': 'microsoft/deberta-v3-large',
 'lr': 1.6e-05,
 'lr_decay': 0.95,
 'weight_decay': 0.03,
 'hidden_dropout_prob': 0.007,
 'attention_probs_dropout_prob': 0.007,
 'n_epochs': 5,
 'batch_size': 2,
 'maxlen': 512,
 'strategy': 'GROUPKFOLD',
 'loss': 'MSELoss',
 'grad_acc_steps': 2,
 'freeze_layers': ['embedding', 'top_half'],
 'save_steps': 100,
 'lightgbm_content_features': ['summary_length',
  'splling_err_num',
  'prompt_length',
  'word_overlap_count',
  'bigram_overlap_count',
  'bigram_overlap_ratio',
  'trigram_overlap_count',
  'trigram_overlap_ratio',
  'quotes_count',
  'pos_mean',
  'unique_word_count',
  'keyword_density',
  'jaccard_similarity',
  'neg_prompt',
  'neu_prompt',
  'pos_prompt',
  'compound_prompt',
  'flesch_reading_ease',
  'count_difficult_words',
  'vocabulary_richness',
  'sentiment_polarity',
  'sentiment_subjectivity',
  'content_pred',
  'wording_pred'],
 'lightgbm_wording_feature

In [22]:
model_dict = {}

for target in targets:
    models = []
    params = study.best_params
    for fold in range(config["n_folds"]):
        X_train = train_df[train_df["fold"] != fold].drop(columns=drop_columns)[config[f"lightgbm_{target}_features"]]
        y_train = train_df[train_df["fold"] != fold][target]
        X_val = train_df[train_df["fold"] == fold].drop(columns=drop_columns)[config[f"lightgbm_{target}_features"]]
        y_val = train_df[train_df["fold"] == fold][target]
        
        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val)
        

        model_params = dict((k,v) for k,v in params.items() if k!="n_features")
        evaluation_results = {}
        model = lgb.train(model_params,
                          num_boost_round=10000,
                          valid_names=['valid', 'train'],
                          train_set=dtrain,
                          valid_sets=[dval,dtrain],
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                              lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        models.append(model)
    
    model_dict[target] = models

[LightGBM] [Warning] Unknown parameter: n_wording_features
[LightGBM] [Warning] Unknown parameter: n_content_features
[LightGBM] [Warning] Unknown parameter: n_wording_features
[LightGBM] [Warning] Unknown parameter: n_content_features
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000940 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3278
[LightGBM] [Info] Number of data points in the train set: 5156, number of used features: 24
[LightGBM] [Warning] Unknown parameter: n_wording_features
[LightGBM] [Warning] Unknown parameter: n_content_features
[LightGBM] [Info] Start training from score -0.039959
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

In [23]:
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []
    
    for fold, model in enumerate(models):
        X_val = train_df[train_df["fold"] == fold].drop(columns=drop_columns)[config[f"lightgbm_{target}_features"]]
        y_val = train_df[train_df["fold"] == fold][target]
    
        
        pred = model.predict(X_val)
        trues.extend(y_val)
        preds.extend(pred)
    
    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.4358338832574363
wording_rmse : 0.569009764805002
mcrmse : 0.5024218240312192


In [24]:
for k,model_list in model_dict.items():
    for i,model in enumerate(model_list):
        model.save_model(f'lightgbm_{k}_fold{i}.txt')

In [25]:
with open("config.json", "w") as outfile:
    json.dump(config, outfile)

# CatBoost

In [26]:
from catboost import CatBoostRegressor

In [27]:
def objective(trial):
    model_dict = {}
    params = {
                "loss_function": 'RMSE',
                'random_state': 42,
                "early_stopping_rounds": 300,
                'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
                'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 2.5), 
                'iterations': trial.suggest_int('iterations', 50, 350),
                "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 50),
                'verbose': False,
                "n_content_features": trial.suggest_int("n_content_features", 18,total_features),
                "n_wording_features": trial.suggest_int("n_wording_features", 18,total_features),
            }

    for target in targets:
        models = []
        fs = SelectKBest(score_func=f_regression, k=params[f"n_{target}_features"])
        fs.fit(train_df.drop(columns=drop_columns), train_df[target])
        selected_features = fs.get_feature_names_out()

        for fold in range(config["n_folds"]):
            X_train = train_df[train_df["fold"] != fold].drop(columns=drop_columns)[selected_features]
            y_train = train_df[train_df["fold"] != fold][target]
            X_val = train_df[train_df["fold"] == fold].drop(columns=drop_columns)[selected_features]
            y_val = train_df[train_df["fold"] == fold][target]
             
            
            model_params = dict((k,v) for k,v in params.items() if k not in ["n_content_features", "n_wording_features"])
            model = CatBoostRegressor(**model_params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)],verbose=False)
            models.append(model)

        model_dict[target] = models
            
    rmses = []

    for target in targets:
        models = model_dict[target]
        fs = SelectKBest(score_func=f_regression, k=params[f"n_{target}_features"])
        fs.fit(train_df.drop(columns=drop_columns), train_df[target])
        selected_features = fs.get_feature_names_out()

        preds = []
        trues = []

        for fold, model in enumerate(models):
            X_val = train_df[train_df["fold"] == fold].drop(columns=drop_columns)[selected_features]
            y_val = train_df[train_df["fold"] == fold][target]
            X_val = X_val[fs.get_feature_names_out()]
            pred = model.predict(X_val)
            trues.extend(y_val)
            preds.extend(pred)

        rmse = np.sqrt(mean_squared_error(trues, preds))
        print(f"{target}_rmse : {rmse}")
        rmses = rmses + [rmse]
        
    return sum(rmses)/len(rmses)

In [28]:
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial), n_trials=250)

[I 2023-10-08 19:48:35,175] A new study created in memory with name: no-name-b2b58bb5-fd36-4f80-8ba8-7bc7a94d63d2
[I 2023-10-08 19:48:40,761] Trial 0 finished with value: 0.5504962087956535 and parameters: {'learning_rate': 0.009675895919461153, 'l2_leaf_reg': 0.0010473754754019118, 'iterations': 188, 'min_data_in_leaf': 37, 'n_content_features': 20, 'n_wording_features': 26}. Best is trial 0 with value: 0.5504962087956535.


content_rmse : 0.47938854930827396
wording_rmse : 0.6216038682830332


[I 2023-10-08 19:48:43,363] Trial 1 finished with value: 0.6111296992372406 and parameters: {'learning_rate': 0.01595481788258747, 'l2_leaf_reg': 7.674975747207396e-07, 'iterations': 76, 'min_data_in_leaf': 38, 'n_content_features': 18, 'n_wording_features': 26}. Best is trial 0 with value: 0.5504962087956535.


content_rmse : 0.5506927941378933
wording_rmse : 0.671566604336588


[I 2023-10-08 19:48:51,980] Trial 2 finished with value: 0.5024349150519123 and parameters: {'learning_rate': 0.0429130619015772, 'l2_leaf_reg': 0.0012885482114831448, 'iterations': 347, 'min_data_in_leaf': 26, 'n_content_features': 20, 'n_wording_features': 19}. Best is trial 2 with value: 0.5024349150519123.


...

content_rmse : 0.4311833976196933
wording_rmse : 0.5692229227511141


[I 2023-10-08 20:24:25,349] Trial 248 finished with value: 0.500203070487907 and parameters: {'learning_rate': 0.04563830593547146, 'l2_leaf_reg': 0.025873044577502512, 'iterations': 330, 'min_data_in_leaf': 42, 'n_content_features': 26, 'n_wording_features': 22}. Best is trial 115 with value: 0.49675026887235185.


content_rmse : 0.43238472672445716
wording_rmse : 0.568021414251357


[I 2023-10-08 20:24:35,270] Trial 249 finished with value: 0.49889859266207104 and parameters: {'learning_rate': 0.04240538326996387, 'l2_leaf_reg': 2.181519667228384e-08, 'iterations': 335, 'min_data_in_leaf': 40, 'n_content_features': 26, 'n_wording_features': 22}. Best is trial 115 with value: 0.49675026887235185.


content_rmse : 0.43020881230787256
wording_rmse : 0.5675883730162695


In [29]:
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(yaxis_range=[0.45,0.55])
fig.show()

In [30]:
params = study.best_params
for target in targets:
    fs = SelectKBest(score_func=f_regression, k=params[f"n_{target}_features"])
    fs.fit(train_df.drop(columns=drop_columns), train_df[target])
    selected_features = fs.get_feature_names_out()
    config[f"catboost_{target}_features"] = list(selected_features)

In [31]:
config

{'name': 'fixed_summary',
 'random_state': 42,
 'n_folds': 4,
 'model_path': 'microsoft/deberta-v3-large',
 'lr': 1.6e-05,
 'lr_decay': 0.95,
 'weight_decay': 0.03,
 'hidden_dropout_prob': 0.007,
 'attention_probs_dropout_prob': 0.007,
 'n_epochs': 5,
 'batch_size': 2,
 'maxlen': 512,
 'strategy': 'GROUPKFOLD',
 'loss': 'MSELoss',
 'grad_acc_steps': 2,
 'freeze_layers': ['embedding', 'top_half'],
 'save_steps': 100,
 'lightgbm_content_features': ['summary_length',
  'splling_err_num',
  'prompt_length',
  'word_overlap_count',
  'bigram_overlap_count',
  'bigram_overlap_ratio',
  'trigram_overlap_count',
  'trigram_overlap_ratio',
  'quotes_count',
  'pos_mean',
  'unique_word_count',
  'keyword_density',
  'jaccard_similarity',
  'neg_prompt',
  'neu_prompt',
  'pos_prompt',
  'compound_prompt',
  'flesch_reading_ease',
  'count_difficult_words',
  'vocabulary_richness',
  'sentiment_polarity',
  'sentiment_subjectivity',
  'content_pred',
  'wording_pred'],
 'lightgbm_wording_feature

In [32]:
model_dict = {}

for target in targets:
    models = []
    params = study.best_params
    for fold in range(config["n_folds"]):
        X_train = train_df[train_df["fold"] != fold].drop(columns=drop_columns)[config[f"catboost_{target}_features"]]
        y_train = train_df[train_df["fold"] != fold][target]
        X_val = train_df[train_df["fold"] == fold].drop(columns=drop_columns)[config[f"catboost_{target}_features"]]
        y_val = train_df[train_df["fold"] == fold][target]
        

        model_params = dict((k,v) for k,v in params.items() if k not in ["n_content_features", "n_wording_features"])
        model = CatBoostRegressor(**model_params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)],verbose=False)
        evaluation_results = {}
        models.append(model)
    
    model_dict[target] = models

In [33]:
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []
    
    for fold, model in enumerate(models):
        X_val = train_df[train_df["fold"] == fold].drop(columns=drop_columns)[config[f"catboost_{target}_features"]]
        y_val = train_df[train_df["fold"] == fold][target]
    
        
        pred = model.predict(X_val)
        trues.extend(y_val)
        preds.extend(pred)
    
    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.4310734856384375
wording_rmse : 0.56859890744486
mcrmse : 0.49983619654164874


In [34]:
for k,model_list in model_dict.items():
    for i,model in enumerate(model_list):
        model.save_model(f'cat_boost_{k}_fold{i}')

In [35]:
with open("config.json", "w") as outfile:
    json.dump(config, outfile)